### Sedaro Ephemeris Demo

This notebook demonstrates how to upload an ephemeris file and use it to describe the position of a spacecraft or other object in Sedaro. We will use examples of the following types of ephemerides:

SPICE SPK - Ephemeris format used by SPICE, a free information system created and maintained by NASA's Navigation and Ancillary
    Information Facility (NAIF). SPICE ephemerides take the format of binary SPK kernels with the file extension .bsp.
    More information can be found here:
    https://naif.jpl.nasa.gov/naif/spiceconcept.html

STK Ephemeris - STK ephemerides are stored in ASCII text files with extension .e. More information can be found here:
    https://help.agi.com/stk/11.0.1/Content/stk/importfiles-02.htm

#### Important: Read Before Running

This notebook makes changes to agent and scenario branches indicated in the settings section. Ensure any changes to the target branches are saved prior to running this code. Sedaro recommends committing current work and creating new branches in the target repositories to avoid loss of work.

This notebook also requires that you have previously generated an API key in the web UI. That key should be stored in a file called `secrets.json` in the same directory as this notebook with the following format:

```json
{
    "API_KEY": "<API_KEY>"
}
```

API keys grant full access to your repositories and should never be shared. If you think your API key has been compromised, you can revoke it in the user settings interface on the Sedaro website.

#### Imports

In [ ]:
import json
import requests

from sedaro import SedaroApiClient
from datetime import datetime, timezone
from tempfile import TemporaryDirectory


#### User Settings

We'll configure some variables to choose the ephemeris files, scenario branch, scenario start/stop, etc.

In [ ]:
SCENARIO_BRANCH_ID = ''                                                # Scenario branch ID where we'll use the ephemeris
AGENT_ID = ''                                                          # ID of the scenario agent that will use the ephemeris. If none, we'll create a peripheral agent for this purpose
NEW_AGENT_NAME = 'Ephemeris Demo Agent'                                # Name of the agent that will be created if AGENT_ID is None

#### Instantiate Client
Instantiate the `SedaroApiClient` with our `API_KEY` and `HOST`.

In [ ]:
# Settings
with open('./secrets.json', 'r') as file:
    API_KEY = json.load(file)['API_KEY']

HOST = "https://api.sedaro.com"
sedaro = SedaroApiClient(api_key=API_KEY, host=HOST)


Once we upload the ephemeris data below, you should be able to simulate if the start and stop time of your sim are configured correctly!  If they're not, we can set them with the SCENARIO_START and SCENARIO_END datetime objects created above, or you can set them using our UI.

In [ ]:
mjd_1900 = 15020.0
dt_1900 = datetime(1900, 1, 1, 0, 0, 0, tzinfo=timezone.utc)

def datetime_to_mjd(dt:datetime) -> float:
    return (dt - dt_1900).total_seconds() / 86400.0 + mjd_1900


def update_start_stop(scenario, start:datetime, stop:datetime):
    scenario.ClockConfig.get_first().update(startTime=datetime_to_mjd(start), stopTime=datetime_to_mjd(stop))

### SPICE Ephemeris

Here we'll use a user supplied SPICE file or download one from NAIF.

In [ ]:
# Settings specific to this demo
SPICE_FILENAME = None # Path to the SPICE SPK you'd like to use (.bsp). If none, use my choice below :)
BACKUP_URL = 'https://naif.jpl.nasa.gov/pub/naif/APOLLO/kernels/spk/apollo15-1.bsp' # or you can edit this url
OBJECT_NAME = '-915' # designator of the object as defined in the supplied file
SCENARIO_START = datetime(1971, 7, 30, 21, 0, 0, tzinfo=timezone.utc)  # Start of the scenario
SCENARIO_END = datetime(1971, 8, 1, 14, 0, 0, tzinfo=timezone.utc)     # End of the scenario

In [ ]:
# Use the client to fetch the scenario branch
scenario = sedaro.scenario(SCENARIO_BRANCH_ID)

# Create the block that contains the ephemeris kinematics. For now the data is missing, but we'll add it later.
ephem_block = scenario.SpkEphemeris.create(objectName=OBJECT_NAME)

# Associate the block with the agent we've chosen, or create a new agent for the purpose
if AGENT_ID:
    agent = scenario.Agent.get(AGENT_ID)
    agent.update(kinematics=ephem_block.id)
    print(f'Updated agent {agent.name} with kinematics {ephem_block.id}.')
else:
    agent = scenario.PeripheralSpacePoint.create(name=NEW_AGENT_NAME, kinematics=ephem_block.id)
    print(f'Created new agent {NEW_AGENT_NAME} with kinematics {ephem_block.id}.')

In [ ]:
# Upload the ephemeris file
route = f'https://sedaro.com/models/branches/{SCENARIO_BRANCH_ID}/ephem/{ephem_block.id}' # This is where we'll post the file

headers = {'X-Api-Key': API_KEY} # This will authenticate the request

# Use the file provided or temporarily download one from NAIF
if SPICE_FILENAME:
    files = {'ephemFile': open(SPICE_FILENAME, 'rb')}
else:
    with TemporaryDirectory() as temp_dir:
        print(f'Downloading from {BACKUP_URL}...')
        with open(f'{temp_dir}/ephem.bsp', 'wb') as file:
            file.write(requests.get(BACKUP_URL).content)
        print('Done!')
        files = {'ephemFile': open(f'{temp_dir}/ephem.bsp', 'rb')}

# Post the file
response = requests.post(route, headers=headers, files=files)

if response.status_code != 200:
    print(f'Error: {response.status_code}')
    print(response.text)

In [ ]:
# (Optional) Set start and end time to the specified values
update_start_stop(scenario, SCENARIO_START, SCENARIO_END)

### STK Ephemeris

I'll use an example from AGI, but we support any Earth-centered EphemerisTimePos or EphemerisTimePosVel. You can get the ephemeris file I'm using here:
https://help.agi.com/stk/#stk/example_ephemeris3.htm

In [ ]:
# Settings specific to this demo
EPHEM_FILENAME = 'example_ephemeris.e' # Path to the .e file
SCENARIO_START = datetime(1971, 7, 30, 21, 0, 0, tzinfo=timezone.utc)  # Start of the scenario
SCENARIO_END = datetime(1971, 8, 1, 14, 0, 0, tzinfo=timezone.utc)     # End of the scenario

In [ ]:
# Use the client to fetch the scenario branch
scenario = sedaro.scenario(SCENARIO_BRANCH_ID)

# Create the block that contains the ephemeris kinematics. For now the data is missing, but we'll add it later.
ephem_block = scenario.StkEphemeris.create()

# Associate the block with the agent we've chosen, or create a new agent for the purpose
if AGENT_ID:
    agent = scenario.Agent.get(AGENT_ID)
    agent.update(kinematics=ephem_block.id)
    print(f'Updated agent {agent.name} with kinematics {ephem_block.id}.')
else:
    agent = scenario.PeripheralSpacePoint.create(name='Ephemeris Demo', kinematics=ephem_block.id)
    print(f'Created new agent {NEW_AGENT_NAME} with kinematics {ephem_block.id}.')


In [ ]:
# Upload the ephemeris file
route = f'https://sedaro.com/models/branches/{SCENARIO_BRANCH_ID}/ephem/{ephem_block.id}' # This is where we'll post the file

headers = {'X-Api-Key': API_KEY} # This will authenticate the request

files = {'ephemFile': open(EPHEM_FILENAME, 'rb')}

# Post the file
response = requests.post(route, headers=headers, files=files)

if response.status_code != 200:
    print(f'Error: {response.status_code}')
    print(response.text)

In [ ]:
# (Optional) Set start and end time to the specified values
update_start_stop(scenario, SCENARIO_START, SCENARIO_END)